In [1]:
import ee
ee.Initialize()


import os
import sys

#Get the absolute path to the parent of current working directory 
cwd    = os.getcwd()
parent = os.path.abspath(os.path.join(cwd, os.pardir))
source_path = os.path.join(parent, 'sources')
sys.path.append(source_path)

sys.path


import geemap
import ImgMask
import Image
import ImgSet
import Mosaic
import LEAFNets

import LEAF_LS as LFLS
import eoAuxData as eoAD
import eoTileGrids as eoTileGD

params = {'sensor': 'L8_SR',
          'unit': 2, 
          'year': 2020,
          'nYears': 1,
          'month': 7,
          'tile_name': 'tile42',
          'prod_name': 'LAI',
          'buff_radius': 10, 
          'scale': 30,   
          'tile_scale': 4,
          'location': 'Drive',
          'folder': 'test_samples'
         }

## Test LEAF Tool with Landsat8/9 Data

In [2]:
region    = eoTileGD.PolygonDict.get(params['tile_name'])  
partition = eoAD.get_GlobLC(region, int(params['year']), True)   #create a biome map
mosaic    = Mosaic.LEAF_Mosaic(params, region)

task_list = []
resultImage = LEAFNets.LS_separate_params(params, mosaic, region, partition, task_list)
print('bands in lai result = ', resultImage.getInfo())

#methodName    = "NAIVE"
#treeDirectory = "users/rfernand387/modisLandsatTrees/"

#method = LFLS.constructMethod(methodName, treeDirectory)

#mosaic = mosaic.addBands(LFLS.predictRF(mosaic, test_method).multiply(1000).round().rename('childNames')) 
#print('\n\n bands in mosaic = ', mosaic.bandNames().getInfo())

'''
[10298, 1149, 1171, 1282, 1324, 1345, 1348, 1481, 1507, 1514, 1676, 1688, 1789, 1949, 2056, 2083, 2085, 2128, 2168, 2187, 2258, 2382, 2483, 2669, 2680, 2713, 2818, 2866, 2959, 2968, 2976, 3089, 3187, 3254, 3265, 3304, 3556, 3714, 3886, 4113, 4171, 4542, 4830, 4857, 4862, 5484, 5600, 5748, 6439, 6473, 7182, 7456]

'''
'''
child_name1 = ee.Number(2056)
child_name2 = ee.Number(1282)

mask1 = mosaic.select('childNames').eq(child_name1)
mask2 = mosaic.select('childNames').eq(child_name2)

child_list = ee.List(ee.FeatureCollection(test_method).get('childFCList'))
used_childFC1 = child_list.filter(ee.Filter.stringContains('system:id', child_name1.format("%d"))).get(0)
used_childFC2 = child_list.filter(ee.Filter.stringContains('system:id', child_name2.format("%d"))).get(0)
print('\n\n size of filtered list = ', used_childFC1.getInfo())

estimate1 = LFLS.predictRF(mosaic.updateMask(mask1), used_childFC1).multiply(1000).toInt16().rename('estimate')
estimate2 = LFLS.predictRF(mosaic.updateMask(mask2), used_childFC2).multiply(1000).toInt16().rename('estimate')

resultImage = LFLS.estimateResponse(test_method, mosaic, region)
print('resultImage = ', resultImage.getInfo())

export_dict = {'image': resultImage,
               'folder': 'LS8_BioParam_map',
               'description': 'LS8_LAI_map',
               'fileNamePrefix': 'LS8_LAI_map',
               'scale': 30,
               'crs': 'EPSG:3979',
               'maxPixels': 1e11,
               'region': region}


task = ee.batch.Export.image.toDrive(**export_dict)
task.start()
'''


<get_LC_map> Used LC map is:  projects/ccmeo-ag-000007/assets/CanLC2020


<LS_separate_params> property names in used method =  ['childFCList', 'RF', 'system:version', 'system:id', 'system:asset_size']
<LS_separate_params> unique biome values =  [0, 1, 10, 2, 3, 5, 6, 7, 9]
bands in lai result =  {'type': 'Image', 'bands': [{'id': 'estimate', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [19, 12], 'origin': [-125, 48], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:footprint': {'type': 'Polygon', 'coordinates': [[[-124.2, 56.439], [-118.75, 48.9443], [-106.596, 51.1826], [-109.537, 59.2506], [-124.2, 56.439]]]}}}


'\nchild_name1 = ee.Number(2056)\nchild_name2 = ee.Number(1282)\n\nmask1 = mosaic.select(\'childNames\').eq(child_name1)\nmask2 = mosaic.select(\'childNames\').eq(child_name2)\n\nchild_list = ee.List(ee.FeatureCollection(test_method).get(\'childFCList\'))\nused_childFC1 = child_list.filter(ee.Filter.stringContains(\'system:id\', child_name1.format("%d"))).get(0)\nused_childFC2 = child_list.filter(ee.Filter.stringContains(\'system:id\', child_name2.format("%d"))).get(0)\nprint(\'\n\n size of filtered list = \', used_childFC1.getInfo())\n\nestimate1 = LFLS.predictRF(mosaic.updateMask(mask1), used_childFC1).multiply(1000).toInt16().rename(\'estimate\')\nestimate2 = LFLS.predictRF(mosaic.updateMask(mask2), used_childFC2).multiply(1000).toInt16().rename(\'estimate\')\n\nresultImage = LFLS.estimateResponse(test_method, mosaic, region)\nprint(\'resultImage = \', resultImage.getInfo())\n\nexport_dict = {\'image\': resultImage,\n               \'folder\': \'LS8_BioParam_map\',\n               \

## Display Tested Results

In [3]:
#color_palette = import('users/gena/packages:palettes').misc.jet[7]

LC_vis = {
'min': 0,
'max': 10000,
'palette': ['000000', '006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5', '555555', '999999', 'CCCCCC', 'FF0000', '00FF00', '0000FF']}

LS_vis = {
 'min': 0,
 'max': 40000,
 'gamma': 1.8,
 'bands': ['SR_B5', 'SR_B4', 'SR_B3']}

S2_vis = {
 'min': 0,
 'max': 6000,
 'gamma': 1.8,
 'bands': ['B8A', 'B4', 'B3']}



Map = geemap.Map(center=[45.41, -75.72], zoom=4)
Map

Map(center=[45.41, -75.72], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childr…

In [4]:
#Map.addLayer(training_set, {'color': 'FFFFFF'}, 'bad samples')
#estimate = estimate1.unmask().clip(region).max(estimate2.unmask().clip(region))

#Map.addLayer(estimate1, LC_vis, 'estimate1 map')
#Map.addLayer(estimate2, LC_vis, 'estimate2 map')
Map.addLayer(resultImage, LC_vis, 'estimate map')
#Map.addLayer(partition, LC_vis, 'biome map')
#Map.addLayer(lai_result, LC_vis, 'leaf map')

#Map.addLayer(valid_mask, {'min': 0, 'max': 1, 'palette': ['000000', '00FF00']}, 'valid mask')
#Map.addLayer(clear_mask, {'min': 0, 'max': 1, 'palette': ['000000', '0000FF']}, 'clear mask')
#Map.addLayer(mask, {'min': 0, 'max': 1, 'palette': ['000000', 'FF0000']}, 'mask')
#Map.addLayer(mask, LC_vis, 'mask image')

#old_LC_map = LC_map.where(LC_map.neq(ee.Image(17)), ee.Image(0))


#Map.addLayer(median_blue, {'min': 0, 'max': 40}, 'blue mosaic')

#Map.addLayer(mosaic, LS_vis, 'LS mosaic')

#Map.addLayer(classified, {'min': 0, 'max': 1, 'palette': ['000000', '00FF00']}, 'new urban map')

#Map.addLayer(st_blocks, LC_vis, 'street block')

#Map.addLayer(ON_builds, {'color': 'FFFFFF'}, 'buildings')

#Map.addLayer(ON_land_use, {'color': 'FFFFFF'}, 'land use')